In [ ]:
#Import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
from scipy.stats import linregress

In [ ]:
#Read the csv files
olympics_df = pd.read_csv('Resources/athlete_events.csv')
regions_df = pd.read_csv('Resources/noc_regions.csv')
host_cities_df = pd.read_csv('Resources/olym.csv')
arrivals_df = pd.read_csv('Resources/INT.ARVL.csv')
departures_df = pd.read_csv('Resources/INT.DPRT.csv')
expenditures_df = pd.read_csv('Resources/INT.XPND.csv')

In [ ]:
#Jumaan start here (Question 1)

In [ ]:
#Jumaan end here

In [ ]:
#Timmy start here (Question 2)

Does hosting olympics affects the travel habit of the people of that country? ( People see lots of different cultures coming to them and want to explore those cultures by themselves) (Timmy)

In [ ]:
olympics_df = olympics_df.replace(['Antwerpen', 'Torino', 'Athina', 'Roma', 'Moskva', 'Sankt Moritz'], ['Antwerp', 'Turin', 'Athens', 'Rome', 'Moscow', 'St. Moritz'])

In [ ]:

host_cities_df = host_cities_df[["City", "Country"]]
host_cities_df.head()

In [ ]:
olympics_country_df = pd.merge(olympics_df, host_cities_df, how="left", on=["City"])
olympics_country_df=olympics_country_df.drop_duplicates()
# remove duplicates
olympics_country_df.head()

In [ ]:
#this allows me to focus on each city that hosted the games, for future merging
cities_only = olympics_country_df.drop_duplicates(subset = ['City'])
cities_only.head()

In [ ]:
#check if it's the correct number
len(cities_only)

In [ ]:
#the only columns important to me
edited_cities_df = cities_only[["City", "Country","Year", "Season"]]
edited_cities_df = edited_cities_df.reset_index(drop=True)


In [ ]:
games_with_data = edited_cities_df.loc[edited_cities_df['Year'] > 1960]


In [ ]:
departures_df.columns

In [ ]:
#basically there is no data til 1995 for departures which is a bummer, so we are dropping coloumns
departures_df.drop(departures_df.iloc[:, 4:39], inplace = True, axis = 1)
departures_df.head()

In [ ]:
#rename coloumn for future merging
departures_df
departures_df = departures_df.rename(columns={'Country Name': 'Country'})

In [ ]:
#change country names in data so it matches
games_with_data = games_with_data.replace(['Russia', 'West Germany', 'South Korea', 'Soviet Union'],
                                          ['Russian Federation', 'Germany', 'Korea, Rep.', 'Russian Federation'])


In [ ]:
#reseting the index for finding specific rows
games_with_data = games_with_data.reset_index(drop=True)

In [ ]:
#no data on Yugoslavia so bye
games_with_data = games_with_data.drop([16])


In [ ]:
#merging data
olympics_host_depart_df = pd.merge(games_with_data, departures_df, how="left", on=["Country"])
olympics_host_depart_df = olympics_host_depart_df.drop_duplicates()

In [ ]:
#departure information basically starts at 1995
olympics_hostingclose_df=olympics_host_depart_df.loc[olympics_host_depart_df['Year'] > 1995]


In [ ]:
#the cities/countries that hosted the olympics and has depart info on them
olympics_hostingclose_df = olympics_hostingclose_df.reset_index(drop=True)
olympics_final_df = olympics_hostingclose_df.drop([9])
olympics_final_df

In [ ]:
olympics_final_df.set_index('Country')


In [ ]:
# starting to try to work data in a way thats good for graphs
for_graph = olympics_final_df

In [ ]:
#simplify drop a bunch of coloums
for_graph = for_graph.drop(['City', 'Year','Season', 'Country Code', 'Indicator Name',
       'Indicator Code', '2019', '2020'], axis=1)

In [ ]:
for_graph.set_index('Country')

In [ ]:
for_graph = for_graph.astype(dict.fromkeys(for_graph.columns[1:25], float))

In [ ]:
#figureout how to do a for loop for this
# do pct chabfe for each and also fill in the first cell since it will be NA

#summer game
uk = for_graph.iloc[0]
uk_years = uk[1 : 25]
uk_pct_hoststart = uk[18: 22].pct_change() * 100 
uk_pct_hoststart = uk_pct_hoststart.fillna(0)

#winter game
usa_1 = for_graph.iloc[1]
usa_1_years = usa_1[1 : 25]
usa_1_pct_hoststart = usa_1[8: 12].pct_change() * 100
usa_1_pct_hoststart = usa_1_pct_hoststart.fillna(0)

#summer game
aus = for_graph.iloc[2]
aus_years = aus[1 : 25]
aus_pct_hoststart = aus[6 : 10].pct_change() * 100
aus_pct_hoststart = aus_pct_hoststart.fillna(0) 

#summer game
usa_2 = for_graph.iloc[3]
usa_2_years = usa_2[1 : 25]
usa_2_pct_hoststart = usa_2[2: 6].pct_change() * 100
usa_2_pct_hoststart = usa_2_pct_hoststart.fillna(0)

#winter game
rus = for_graph.iloc[4]
rus_years = rus[1 : 25]
rus_pct_hoststart = rus[20 : 24].pct_change() * 100
rus_pct_hoststart = rus_pct_hoststart.fillna(0)

#winter game
jap = for_graph.iloc[5]
jap_years = jap[1 : 25]
jap_pct_hoststart = jap[4 : 8].pct_change() * 100
jap_pct_hoststart = jap_pct_hoststart.fillna(0)

#winter game
ita = for_graph.iloc[6]
ita_years = ita[1 : 25]
ita_pct_hoststart = ita[12 : 16].pct_change() * 100
ita_pct_hoststart = ita_pct_hoststart.fillna(0)

#summer game
chi = for_graph.iloc[7]
chi_years = chi[1 : 25]
chi_pct_hoststart = chi[14 : 18].pct_change() * 100
chi_pct_hoststart = chi_pct_hoststart.fillna(0)

#summer game
bra = for_graph.iloc[8]
bra_years = bra[1 : 25]
bra_pct_hoststart = bra[22 : 26].pct_change() * 100
bra_pct_hoststart = bra_pct_hoststart.fillna(0)

#winter game
can = for_graph.iloc[9]
can_years = can[1 : 25]
can_pct_hoststart = can[16 : 20].pct_change() * 100
can_pct_hoststart = can_pct_hoststart.fillna(0)


In [ ]:
#pecercent change graph of when each country hosted the oylmpics
x_axis = range(len(uk_pct_hoststart)) 


plt.plot(x_axis, ita_pct_hoststart)
plt.plot(x_axis, can_pct_hoststart)
plt.plot(x_axis, usa_1_pct_hoststart)
plt.plot(x_axis, jap_pct_hoststart)
plt.plot(x_axis, rus_pct_hoststart)

plt.legend(['Italy 2006','Canada 2010','USA 2002','Japan 1998', 'Russia 2014'],
           loc='center left', bbox_to_anchor=(1, 0.82))

plt.title("Winter Olympic Games")
plt.ylabel("Percent Change %")
plt.xlabel("Years Since Hosting")
plt.savefig("graphs_Timmy/pct_change_winter")
plt.show()

In [ ]:
#fillin in the last value because it isnt worthing throuwing out that row
bra_pct_hoststart["2019"] = np.nan
bra_pct_hoststart = bra_pct_hoststart.fillna(method="ffill")

In [ ]:
#pecercent change graph of when each country hosted the oylmpics
plt.plot(x_axis, uk_pct_hoststart)
plt.plot(x_axis, aus_pct_hoststart)
plt.plot(x_axis, usa_2_pct_hoststart)
plt.plot(x_axis, chi_pct_hoststart)
plt.plot(x_axis, bra_pct_hoststart)

plt.legend(['United Kingdom 2012','Australia 2000','USA 1996','China 2008', 'Brazil 2016'],
           loc='center left', bbox_to_anchor=(1, 0.82))

plt.title("Summer Olympic Games")
plt.ylabel("Percent Change %")
plt.xlabel("Years Since Hosting")
plt.savefig("graphs_Timmy/pct_change_summer")
plt.show()

In [ ]:
years = [int(i) for i in for_graph.iloc[:,1:].columns.tolist()]
years_early = [int(i) for i in for_graph.iloc[:,1:].columns.tolist()]

#years.drop(years.index[[1,2,3,4,5,6,7,8,9,10,11]], inplace=True)
#del years[0:13]
del years_early[13:25]


In [ ]:
china = for_graph.iloc[7]
china_postyears = china[14 : 25]
china_preyears = china[1 : 14]
china_pct_hoststart = china[13 : 25].pct_change() * 100


In [ ]:
china_preyears = china_preyears.astype(float)
china_postyears = china_postyears.astype(float)



In [ ]:
#neccsary
years = years[0:24]
    
years_ints = []
for item in years:
    years_ints.append(int(item))

china = china[1:25]

china_ints = []
for x in china:
    china_ints.append(int(x))

In [ ]:
#when in doubt put it in a new dataframe, helpgul tip the rows must match
china_df = pd.DataFrame({"Total Years":years_ints, 
                        "Total Departs":china_ints})



In [ ]:


early_years = china_df['Total Years'][0:13]
early_departs = china_df['Total Departs'][0:13]

post_years = china_df['Total Years'][14:24]
post_departs = china_df['Total Departs'][14:24]


In [ ]:
x_values = china_df['Total Years']
y_values = china_df['Total Departs']


plt.scatter(x_values, y_values, marker = "o", color = "blue",edgecolor = "black")

#overall
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#early years1
(slope1, intercept1, rvalue1, pvalue1, stderr1) = linregress(early_years, early_departs)
regress_values1 = early_years * slope1 + intercept1
line_eq1 = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#later years 2
(slope2, intercept2, rvalue2, pvalue2, stderr2) = linregress(post_years, post_departs)
regress_values2 = post_years * slope2 + intercept2
line_eq2 = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


#plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"g-")
plt.plot(early_years,regress_values1,"r-")
plt.plot(post_years,regress_values2,"b-")

plt.grid()
plt.xlabel("Years")
plt.ylabel("Departures from China(mil)")
plt.title("Trend Line")
plt.savefig("graphs_Timmy/China_rate_change")
plt.show()

In [ ]:
round(slope2)/round(slope1)

   The Olympics is a massive international event that brings together around two hundred countries for an exciting 
two weeks of athletic events. However, our thesis states that hosting the Olympics has much bigger effects that 
go past just the two week period. My section is focused on how it impacts the citizens of the host country, 
specifically in the focus of tourism. We chose to focus on tourism because one of the underlying messages of 
the Olympics is of unity, of exploration, and to push oneself. This emphasis is more hypothetically more heavily felt 
by the host countries citizens that are seeing the best of their own country on this international stage. The impact 
should be a noticeable increase in citizens going on international trips to visit other countries, since they were 
inspired to explore the world. In the following paragraph we shall see if this holds up, and how I worked through the
data to find said results.
   I’ve learned that cleaning the data, and completely understanding is not only fundamental to analyzing data, but also fascinating. The routes each of us took and branched from to find answers for each of our own questions shows the flexibility and the power of this first crucial step to analyzing any data. I digress, the main highlights, are whittling down the massive olympics data to only have the city, the country, the year, and season of the game for the past hundred years, which is forty-two places. Then by merging left and constricting for the years on when we have information on the number of departures, which results to ten cities from 1995 to 2018. Perfectly, that leaves us with five Winter Olympic Games and five Summer Olympic Games to analyze. To graph some of the data I discovered. I focused one section at looking at all the summer and winter games on their own graphs, and I believe the best way to visualize the changes over time would be a multi line graph to compare the trajectories in a succinct way. The best way is to have the year of each country hosting as the start date and then make the each timeline relative to it’s own growth, so I decided to make this graph focus on the percent change for the next three years to see if there was an increase or decrease in departures. What I found seems to be slightly positive to an increase in departures post hosting the Olympics, however it’s more inconclusive, since the results are generally all over the place without any similar patterns matching up. Which is frustrating, but fair since, the world is a complex place and not everything is going to be so simple for a cause and effect.
   The second graph, I decided to focus on one country and see if a linear regression could be insightful to see if the rate of departures changed from pre hosting the Olympics to post. I chose China as the candidate because in just a few months it will be hosting the Olympics again, so it would be fun to see if any the linear regression would still hold true. (Though with the pandemic about, probably not.) I created the linear regressions and found the slope after hosting was three point nine times the rate of departures before hosting the Olympics. This could be a strong result for use to be able to slightly reject the null hypothesis.
   This increase of international tourism and travel can be from the impact the Olympic Games had on individual people as well as corporations to pursue connections outside of China. As China is one of the booming economies the more business fields the countries companies can partner with the more travel to said countries would most likely follow. That is just one of several reasons why the departure rate increased, still I’m happy that we can reject the null hypothesis in some capacity!

In [ ]:
#Timmy end here

In [ ]:
#Raj start here (Question 3)

In [ ]:
#Raj end here

In [ ]:
#Fatma start here (Question 4)

In [ ]:
#Fatma end here